## 저장내용 불러오기

In [106]:
import pandas as pd

train_df = pd.read_csv('elect_training.csv', header=None, )
print(train_df[1].value_counts())
test_df = pd.read_csv('elect_test.csv', header=None)
print(test_df[1].value_counts())

0    309
1    297
Name: 1, dtype: int64
1    209
0    200
Name: 1, dtype: int64


## 단어 토큰화

In [101]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

## 모델 생성

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df[0])
tfidf_matrix_train = tfidf_vect.transform(train_df[0])
tfidf_matrix_test = tfidf_vect.transform(test_df[0])

## 모델학습

In [64]:
import xgboost as xgb
from xgboost import plot_importance
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [109]:
# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_wrapper.fit(tfidf_matrix_train, train_df[1])

w_preds = xgb_wrapper.predict(tfidf_matrix_test)

In [118]:
print('XGBoost 정확도: ', accuracy_score(test_df[1], w_preds))

XGBoost 정확도:  0.9706601466992665
